In [ ]:
#Import packages

import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import statsmodels.api as sm
import math
from sklearn.neural_network import MLPClassifier

import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import statsmodels.api as sm
import math

import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics
from sklearn import linear_model, datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
# from sklearn.cross_validation import cross_val_predict
from sklearn.svm.libsvm import cross_validation
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

import scipy.stats

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Import and clean Republiq data set
data = pd.read_excel("data_stedin_excel_min.xlsx")
data = data[data['aangepakt'] == data['aangepakt']]

# Import and clean surface area data set
data2 = pd.read_csv("verbruik_data_met_oppervlakte.csv")
data2 = data2[data2['aangepakt'] == data2['aangepakt']]

In [ ]:
def analysis(data, iv, kleine_c, lasso, energie, data2, oppervlakte):
    
    X = data.copy()
    y = data2[energy]
    
    cols = ['Retrofitted', 'Apartment', 'Semi-detached', 'Detached', 'Corner_house', '-1945','1945-1964', '1965-1974', '1991-2005', '2006-']
    X = X[cols]
    
    # Construct interaction variables
    if iv == 1:
        temp = []
        temp2 = []

        for i in columns:
            for j in columns:
                if j not in temp2:
                    temp.append([i,j])
            temp2.append(i)
            
        columns2 = columns.copy()

        for i in temp:
            columns2.append(i)
            
        for i in columns2:
            if len(i) == 2:
                X[i[0] + ' x ' + i[1]] = X[i[0]] * X[i[1]]
    
    # Include surface are
    if (oppervlakte == 1) and (kleine_c == 1):
        data = data[data['categorie_match'] == 'PC6']
    
    y_column = [energie]
    x_columns = ["woningtype_naam", "woningtype_bouwperiode", "woningtype_daktype", "aangepakt"]
    
    # Include small cluster
    if (kleine_c == 1) and (oppervlakte == 0):
        data = data[data['groepsgrootte'] < 20]
    
    X = data[x_columns]
    X = pd.get_dummies(X)
    y = data[y_column]
    
    # Include both small clusters and surface are
    if (kleine_c == 1) and (oppervlakte == 1):
        X['Detached'] = 0
        cols = ['Retrofitted', 'Semi-detached', 'Apartment', 'Corner_house', 'Town_house', '1945-1964', '1965-1974', '1975-1991', '1991-2005', '2006-', '-1945', 'Flat_roof', 'Sloped_roof', 'Detached']
        X.columns = cols
        cols = ['Retrofitted', 'Apartment', 'Semi-detached', 'Detached', 'Corner_house', 'Town_house', '-1945','1945-1964', '1965-1974', '1975-1991', '1991-2005', '2006-', 'Flat_roof', 'Sloped_roof']
        X = X[cols]
    else:
        cols = ['Retrofitted', 'Semi-detached', 'Apartment', 'Corner_house', 'Town_house', 'Detached', '1945-1964', '1965-1974', '1975-1991', '1991-2005', '2006-', '-1945', 'Flat_roof', 'Sloped_roof']
        X.columns = cols
        cols = ['Retrofitted', 'Apartment', 'Semi-detached', 'Detached', 'Corner_house', 'Town_house', '-1945','1945-1964', '1965-1974', '1975-1991', '1991-2005', '2006-', 'Flat_roof', 'Sloped_roof']
        X = X[cols]

    # Create train/ test split
    list1 = set(X.index)

    train = sample(list1, int(0.7 * len(list1)))

    X_train = X.loc[train, :]
    y_train = y[train]

    train = set(train)

    test = list1 - train

    X_test = X.loc[test, :]
    test = list(test)
    y_test = y[test]
    
    # Initialize alpha values and empty scores arrays
    values = [0, 0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 20, 30]
    scores = []
    scores2 = []
    
    # Run analysis for all alpha values
    for i in values:
        from sklearn import linear_model
        clf = linear_model.ElasticNet(alpha=i, l1_ratio = lasso)

        clf.fit(X_train,y_train)

        preds = list(clf.predict(X_test))
        trues = list(y_test)

        errors = [a_i - b_i for a_i, b_i in zip(preds, trues)]

        rmse = [math.sqrt(x**2) for x in errors]

        scores.append(sum(rmse)/len(rmse))
        
        # R-squared
        coef = clf.coef_
        p = numpy.poly1d(coef)
        
        # Fit values, and mean
        yhat = clf.predict(X_train)

        ybar = numpy.sum(y_train)/len(y_train)
        ssreg = numpy.sum((yhat - ybar)**2)
        sstot = numpy.sum((y_train - ybar)**2)
            
        coef = clf.coef_
        
        # Calculate MSE
        MSE = np.mean((y_test - clf.predict(X_test).T)**2)
        var_est = MSE * np.diag(np.linalg.pinv(np.dot(X_test.T,X_test)))
        SE_est = np.sqrt(var_est)
                
        temp = pd.DataFrame()
        temp['Coef'] = coef
                
        tvalues = coef / SE_est
                
        #find p-value
        pvalues = []
        for k in tvalues:
            pvalues.append(scipy.stats.t.sf(abs(k), df=0.7*len(data)-len(coef)))
                    
        temp['P-value'] = pvalues
                
        length = len(temp[(temp['Coef'] != 0) & (temp['P-value'] < 0.01)])
          
        # Append scores to initialized dataframe    
        scores2.append([i, sum(abs(clf.coef_)), sum(rmse)/len(rmse), length, ssreg / sstot])
    
    # Rerun model with best performing hyperparameters
    max_value = min(scores)
    max_index = scores.index(max_value)

    clf = linear_model.ElasticNet(alpha=values[max_index], l1_ratio = lasso)

    clf.fit(X_train,y_train)

    coef = clf.coef_
            
    MSE = np.mean((y_train - clf.predict(X_train).T)**2)
    var_est = MSE * np.diag(np.linalg.pinv(np.dot(X_train.T,X_train)))
    SE_est = np.sqrt(var_est)
            
    return [coef, SE_est, X, scores2]

In [ ]:
# Run the model for the different data sets
coef, se, X, scores2 = analysis(data, 0, 0, 0, 'SJV_2017_ELEKTRA', data2, 0)
tvalues = coef / se

# Construct characteristics of the model
pvalues = []
for i in tvalues:
    pvalues.append(scipy.stats.t.sf(abs(i), df=0.7*len(data)-len(coef)))

results = pd.DataFrame(index=X.columns)

results['Coefficient'] = coef
results['Standard Error'] = se
results['P-value'] = pvalues

In [ ]:
# Plot the RMSE, Coefficients unequal to zero and total sum of coefficeints for different alpha values
scors = pd.DataFrame(scores2)

fig, ax1 = plt.subplots()

plt.xscale('log')

color = 'tab:red'
ax1.set_xlabel('Alpha')
ax1.set_ylabel('Sum of coefficient', color=color)
ax1.plot(scors[0], scors[1], color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:green'
ax2.set_ylabel('RMSE', color=color)  # we already handled the x-label with ax1
ax2.plot(scors[0], scors[3], color=color)
ax2.tick_params(axis='y', labelcolor=color)

color = 'tab:blue'
ax2.set_ylabel('RMSE', color=color)  # we already handled the x-label with ax1
ax2.plot(scors[0], scors[2], color=color)
ax2.tick_params(axis='y', labelcolor=color)


fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()